In [1]:
from google.cloud import bigquery
from google.cloud import bigquery_storage
creds = "/Users/srdeo/OneDrive - Copart, Inc/secrets/cprtpr-datastewards-sp1-614d7e297848 (1).json"
client = bigquery.Client.from_service_account_json(json_credentials_path=creds)
bqstorageclient = bigquery_storage.BigQueryReadClient(credentials=creds)
from IPython.core.interactiveshell import InteractiveShell
InteractiveShell.ast_node_interactivity = "all"

#### For buyers placing bids:

In [83]:
## For active buyers

query1 = f"""
with filters as
(
select f.mbr_lic_type, f.mbr_state, b.lot_nbr, b.buyer_nbr, f.mbr_email, b.bid_amt, l.inv_dt, l.lot_year, l.lot_make_cd, l.grp_model, l.acv, l.plug_lot_acv, l.repair_cost
from `cprtpr-dataplatform-sp1`.usviews.v_us_bids_fact b
join `cprtpr-dataplatform-sp1`.usviews.v_us_lot_fact l
on b.lot_nbr = l.lot_nbr
join `cprtpr-dataplatform-sp1`.usviews.v_us_member_fact f
on f.mbr_nbr = b.buyer_nbr
where l.inv_dt between '2025-07-26' and '2025-10-27'
and lot_type_cd = 'V' and yard_country_cd = 'USA'
and mbr_country = 'USA' AND mbr_mbrshp_type_cd IN ('BASIC', 'PREMIER')
  AND mbr_site_status_cd = 'A'
AND mbr_mbrshp_status_cd = 'A' AND mbr_lang_pref = 'en'
AND mbr_status = 'A'
),
lots_bid_counts as (
 select lot_nbr, count(distinct buyer_nbr) as total_unique_buyers_on_that_lot
    from filters
    group by 1
 ),
 buyers_bid_counts as (
 select buyer_nbr, count(distinct lot_nbr) as total_unique_lots_bid_by_buyers
 from filters
 group by 1
 )
 select f.mbr_lic_type, f.mbr_state, f.lot_nbr, f.buyer_nbr, f.mbr_email, max(f.bid_amt) as max_bid, f.inv_dt, f.lot_year, f.lot_make_cd, f.grp_model, f.acv, f.plug_lot_acv, f.repair_cost, lb.total_unique_buyers_on_that_lot, bb.total_unique_lots_bid_by_buyers
 from filters f
 left join lots_bid_counts as lb on f.lot_nbr = lb.lot_nbr
 left join buyers_bid_counts as bb on f.buyer_nbr = bb.buyer_nbr
 GROUP BY
    f.mbr_lic_type, f.mbr_state, f.lot_nbr, f.buyer_nbr, f.mbr_email, f.inv_dt, f.lot_year, f.lot_make_cd, f.grp_model,f.plug_lot_acv,
    f.acv, f.repair_cost, lb.total_unique_buyers_on_that_lot, bb.total_unique_lots_bid_by_buyers
"""

data1 = client.query(query1).to_dataframe()

In [84]:
active_buyers = data1.copy()

In [85]:
active_buyers

,mbr_lic_type,mbr_state,lot_nbr,buyer_nbr,mbr_email,max_bid,inv_dt,lot_year,lot_make_cd,grp_model,acv,plug_lot_acv,repair_cost,total_unique_buyers_on_that_lot,total_unique_lots_bid_by_buyers
0,Dealer,NV,62520895,835040,cncauto775@gmail.com,1950.0,2025-07-31,2015,BMW,X1,9778.00,6525.0,0.00,5,332
1,Dealer,NV,80901305,260760,olea8086@gmail.com,3250.0,2025-10-08,2017,ACUR,MDX,16297.00,16025.0,11752.67,5,223
2,Consumer,WY,64241495,845199,jdpcdp7280@gmail.com,15400.0,2025-10-01,2016,CHEV,CORVETTE,35661.00,35125.0,30000.00,11,6
3,Dismantler,NV,69257835,39955,luke@nvpap.com,125.0,2025-09-23,2013,NISS,SENTRA,6172.47,6000.0,6172.47,2,5479
4,Dismantler,NV,57070355,313809,odinaseric@gmail.com,550.0,2025-10-06,1998,TOYT,TACOMA,7474.00,0.0,7474.00,2,390
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3883366,Consumer,WI,62886645,384414,shaffer1213@gmail.com,7900.0,2025-08-06,2013,JEP,WRANGLER,15578.00,14700.0,8073.00,14,1
3883367,Dismantler,MI,68892335,728084,eliteautocenter6095@gmail.com,350.0,2025-08-25,2024,JEP,WRANGLER,49257.00,46350.0,0.00,10,777
3883368,Dealer,LA,61105405,771819,nkwhitney@gmail.com,250.0,2025-08-25,1997,JEP,WRANGLER,6474.00,0.0,6724.91,2,42
3883369,Dismantler,TN,49257965,764700,rhonda.colding@roadtestedparts.com,4700.0,2025-07-29,2012,JEP,WRANGLER,13533.31,12425.0,13533.31,8,6023


In [86]:
active_buyers['buyer_nbr'].nunique()

80766

In [87]:
active_buyers.to_csv('../data/raw/active_buyers.csv',index=False)

In [88]:
active_buyers[active_buyers['buyer_nbr']==156698]

,mbr_lic_type,mbr_state,lot_nbr,buyer_nbr,mbr_email,max_bid,inv_dt,lot_year,lot_make_cd,grp_model,acv,plug_lot_acv,repair_cost,total_unique_buyers_on_that_lot,total_unique_lots_bid_by_buyers
103461,Consumer,IN,57502915,156698,benwmichaels36@icloud.com,125.0,2025-07-23,2013,BMW,5 SERIES,11025.00,8350.0,11025.0,6,8
402369,Consumer,IN,55850725,156698,benwmichaels36@icloud.com,1300.0,2025-08-14,2018,LAND,RANGEROVER,14625.00,14025.0,0.0,37,8
937275,Consumer,IN,59674335,156698,benwmichaels36@icloud.com,950.0,2025-07-18,2016,BMW,X3,4485.00,9325.0,0.0,13,8
1246883,Consumer,IN,58225065,156698,benwmichaels36@icloud.com,850.0,2025-08-05,2024,BMW,M3,59627.55,55725.0,0.0,46,8
1319562,Consumer,IN,61119935,156698,benwmichaels36@icloud.com,3000.0,2025-08-28,2017,BMW,X3,0.00,10775.0,0.0,20,8
1386998,Consumer,IN,57714695,156698,benwmichaels36@icloud.com,500.0,2025-08-01,2012,SUBA,IMPREZA,9899.00,7600.0,0.0,17,8
1438240,Consumer,IN,56826135,156698,benwmichaels36@icloud.com,1150.0,2025-07-16,2011,BMW,5 SERIES,2558.00,5950.0,4897.6,7,8
2890021,Consumer,IN,43960615,156698,benwmichaels36@icloud.com,250.0,2025-08-11,2018,FORD,F250,38885.00,31275.0,37393.0,12,8


#### For buyers placing no bids

In [89]:
## Buyers with no bids

query2 = f"""
SELECT f.mbr_lic_type, f.mbr_state, f.mbr_nbr, f.mbr_email
FROM `cprtpr-dataplatform-sp1`.usviews.v_us_member_fact f
left join `cprtpr-dataplatform-sp1`.usviews.v_us_bids_fact b
ON f.mbr_nbr = b.buyer_nbr
and b.auc_dt between '2025-07-26' and '2025-10-27'
WHERE mbr_mbrshp_type_cd IN ('BASIC', 'PREMIER')
  AND mbr_status = 'A'
  AND mbr_site_status_cd = 'A'
 and bid_id is null
 and mbr_country = 'USA'
 mbr_lang_pref = 'en'
"""

data2 = client.query(query2).to_dataframe()

In [90]:
data2

,mbr_lic_type,mbr_state,mbr_nbr,mbr_email
0,Consumer,WV,761781,Adasher9@outlook.com
1,Consumer,HI,710225,noahriverjohnson@gmail.com
2,Consumer,AK,124085,WAIMAN.FREEFLYING@GMAIL.COM
3,Consumer,NH,252896,cmcgrath09@gmail.com
4,Consumer,DE,727700,dorzetshrt@yahoo.com
...,...,...,...,...
145504,Consumer,WI,236011,erik.steven.hall@gmail.com
145505,Consumer,WI,203256,jofu3@icloud.com
145506,Dealer,WI,191070,info@jmpaintingcontractors.com
145507,Consumer,WI,562736,bennettbrown25@gmail.com


In [91]:
data2['mbr_nbr'].nunique()

145509

In [92]:
data2.to_csv('data/raw/non_active_buyers.csv', index=False)

In [93]:
data2[data2['mbr_nbr']==156698]

,mbr_lic_type,mbr_state,mbr_nbr,mbr_email
54315,Consumer,IN,156698,benwmichaels36@icloud.com


### Future Lots

In [94]:
## Future lots

query5 = """
SELECT
  lot_nbr, lot_year, lot_make_cd, grp_model, damage_type_desc, repair_cost, acv, plug_lot_acv,auc_dt, COALESCE(proquote_amt, proquote_amt_new) AS proquote_amt
FROM `cprtpr-dataplatform-sp1.usviews.v_us_lot_fact` l
WHERE l.auc_dt BETWEEN DATE_ADD(CURRENT_DATE(), INTERVAL 1 DAY) AND DATE_ADD(CURRENT_DATE(), INTERVAL 7 DAY)
  and loss_type_cd not in ('B','K','F')
    and damage_type_cd not in ('BN','BE','BI','BC')
        and lot_type_cd = 'V' and yard_country_cd = 'USA'
  and offsite_flag = 'N'
"""

data5 = client.query(query5).to_dataframe()

In [95]:
data5.to_csv('../data/raw/upcoming_lots.csv', index=False)

In [96]:
data5

,lot_nbr,lot_year,lot_make_cd,grp_model,damage_type_desc,repair_cost,acv,plug_lot_acv,auc_dt,proquote_amt
0,71123705,2020,HOND,PASSPORT,FRONT END,20089.12,23445.00,24225.0,2025-10-21,9536.78
1,62754925,2003,BMW,Z4,FRONT END,3200.00,3247.00,0.0,2025-10-22,1259.12
2,85277025,2020,INFI,QX50,FRONT END,21489.38,25417.00,21100.0,2025-10-22,6788.11
3,67924785,2013,AUDI,Q7,MECHANICAL,0.00,3380.00,6725.0,2025-10-22,858.85
4,86845485,2017,SUBA,WRX,MINOR DENT/SCRATCHES,0.00,0.00,15425.0,2025-10-21,7563.92
...,...,...,...,...,...,...,...,...,...,...
45177,80203955,2017,JEP,WRANGLER,ALL OVER,20755.29,20449.00,22525.0,2025-10-24,6150.38
45178,71829755,2015,JEP,WRANGLER,VANDALISM,0.00,24500.00,16625.0,2025-10-21,12232.06
45179,85198465,2017,JEP,WRANGLER,MECHANICAL,16758.25,19574.96,18175.0,2025-10-24,7409.81
45180,81837765,2006,JEP,WRANGLER,FRONT END,0.00,8806.84,7275.0,2025-10-24,1238.04


## Popular Lots

In [97]:
query6 = """
WITH cte AS (
  SELECT
    l.buyer_type, f.mbr_state, l.lot_make_cd, l.grp_model,COUNT(*) AS cnt,
    APPROX_QUANTILES(l.acv, 2)[OFFSET(1)] AS median_acv, APPROX_QUANTILES(l.plug_lot_acv, 2)[OFFSET(1)] AS median_plug_lot_acv, APPROX_QUANTILES(l.repair_cost, 2)[OFFSET(1)] AS median_repair_cost
  FROM `cprtpr-dataplatform-sp1`.usviews.v_us_member_fact f
  LEFT JOIN `cprtpr-dataplatform-sp1`.usviews.v_us_lot_fact l
    ON f.mbr_nbr = l.buyer_nbr
  WHERE f.mbr_mbrshp_type_cd IN ('BASIC', 'PREMIER')
    AND f.mbr_status = 'A' AND f.mbr_country = 'USA' AND f.mbr_site_status_cd = 'A'
    AND l.inv_dt BETWEEN '2025-07-26' and '2025-10-27'
    AND l.lot_year >= 2017
    AND l.lot_type_cd = 'V' AND l.yard_country_cd = 'USA'
  GROUP BY l.buyer_type, f.mbr_state, l.lot_make_cd, l.grp_model
),

deduped AS (
  SELECT *,
         ROW_NUMBER() OVER (
           PARTITION BY buyer_type, mbr_state, lot_make_cd, grp_model
           ORDER BY cnt DESC
         ) AS model_rank
  FROM cte
),

ranked AS (
  SELECT *,
         ROW_NUMBER() OVER (
           PARTITION BY buyer_type, mbr_state
           ORDER BY cnt DESC
         ) AS rank
  FROM deduped
  WHERE model_rank = 1
)

SELECT *
FROM ranked
WHERE rank <= 6
"""

data6 = client.query(query6).to_dataframe()


In [98]:
data6

,buyer_type,mbr_state,lot_make_cd,grp_model,cnt,median_acv,median_plug_lot_acv,median_repair_cost,model_rank,rank
0,Automotive Related Business,NJ,HOND,ACCORD,2,16141.0,20625.0,11613.00,1,1
1,Automotive Related Business,NJ,HOND,CIVIC,1,21839.0,25050.0,15959.01,1,2
2,Consumer,MO,HYUN,ELANTRA,9,12150.0,12400.0,6582.93,1,1
3,Consumer,MO,NISS,SENTRA,8,8767.0,12975.0,0.00,1,2
4,Consumer,MO,FORD,ESCAPE,7,10316.0,9925.0,0.00,1,3
...,...,...,...,...,...,...,...,...,...,...
1317,General Business,PA,HOND,CRV,54,26264.0,28250.0,18192.76,1,2
1318,General Business,PA,HOND,ACCORD,38,18190.0,23525.0,14899.83,1,3
1319,General Business,PA,NISS,MAXIMA,16,12466.0,14100.0,9444.36,1,4
1320,General Business,PA,CHEV,MALIBU,15,10955.0,13575.0,10155.40,1,5


In [99]:
data6[(data6['buyer_type']=='Consumer') & (data6['mbr_state']=='AL')]

,buyer_type,mbr_state,lot_make_cd,grp_model,cnt,median_acv,median_plug_lot_acv,median_repair_cost,model_rank,rank
879,Consumer,AL,CHEV,SILVERADO,3,51701.00,49800.0,31738.45,1,1
880,Consumer,AL,HOND,CIVIC,3,11332.31,14075.0,11332.31,1,2
881,Consumer,AL,CHEV,CAMARO,2,16660.00,19900.0,2445.03,1,3
882,Consumer,AL,RAM,2500,2,37087.00,41175.0,10388.99,1,4
883,Consumer,AL,NISS,ALTIMA,2,8824.00,11900.0,7628.77,1,5
884,Consumer,AL,CHRY,300,2,12216.00,20950.0,13923.00,1,6


In [100]:
data6['buyer_type'].value_counts()

buyer_type
Dealer                         306
Consumer                       295
Dismantler                     269
General Business               197
Export                         160
Automotive Related Business     52
Name: count, dtype: int64

In [101]:
data6.to_csv('../data/raw/popular_lots.csv', index=False)

### Westake Lots

In [70]:
## Future lots

query7 = """
select lot_nbr, lot_stg, lot_year, lot_make_cd, grp_model, damage_type_desc, repair_cost, acv, plug_lot_acv, auc_dt, COALESCE(proquote_amt, proquote_amt_new) AS proquote_amt
from `cprtpr-dataplatform-sp1`.usviews.v_us_lot_fact where
                                                         seller_nbr = 'WLF2'
                                                     and
                                                         (auc_dt >= '2025-10-17' or auc_dt is null)
                                                     and abnormal_close_type = ''
    and lot_stg > 30
    and loss_type_cd not in ('B','K','F')
    and damage_type_cd not in ('BN','BE','BI','BC')
"""

data7 = client.query(query7).to_dataframe()

In [71]:
data7

,lot_nbr,lot_stg,lot_year,lot_make_cd,grp_model,damage_type_desc,repair_cost,acv,plug_lot_acv,auc_dt,proquote_amt
0,80762855,40,2017,MERZ,C-CLASS,NORMAL WEAR,7261.0,13220.0,15850.0,NaT,0.00
1,85948435,40,2012,GMC,TERRAIN,FRONT END,2783.0,5260.0,6250.0,NaT,0.00
2,85654195,40,2011,HOND,CIVIC,NORMAL WEAR,8092.0,6900.0,7975.0,NaT,879.94
3,80588215,50,2011,GMC,SIERRA,MECHANICAL,11315.0,7440.0,8625.0,NaT,1012.41
4,86334335,40,2017,TOYT,HIGHLANDER,SIDE,30808.0,17100.0,20850.0,NaT,5008.09
...,...,...,...,...,...,...,...,...,...,...,...
319,84297605,50,2019,FORD,EXPLORER,FRONT END,5643.0,21420.0,24850.0,NaT,2359.35
320,84837845,40,2016,CHEV,CRUZE,FRONT END,0.0,6900.0,8600.0,NaT,0.00
321,85654165,40,2020,HYUN,SANTA FE,SIDE,6485.0,14620.0,17525.0,NaT,4508.17
322,85544495,40,2006,LAND,LR3,MINOR DENT/SCRATCHES,835.0,3960.0,4000.0,NaT,909.14


In [79]:
data7[data7['lot_nbr']==84051325]

,lot_nbr,lot_stg,lot_year,lot_make_cd,grp_model,damage_type_desc,repair_cost,acv,plug_lot_acv,auc_dt,proquote_amt
148,84051325,50,2025,NISS,SENTRA,FRONT END,19380.0,22800.0,22800.0,NaT,4065.8


In [72]:
data7['lot_nbr'].nunique()

324

In [73]:
data7.to_csv('data/raw/wl_lots.csv', index=False)

In [22]:
## SO cal buyer

## For active buyers

query8 = f"""
with filters as
(
select  b.buyer_nbr
from `cprtpr-dataplatform-sp1`.usviews.v_us_bids_fact b
join `cprtpr-dataplatform-sp1`.usviews.v_us_lot_fact l
on b.lot_nbr = l.lot_nbr
join `cprtpr-dataplatform-sp1`.usviews.v_us_member_fact f
on f.mbr_nbr = b.buyer_nbr
where l.inv_dt between '2025-07-13' and '2025-10-13'
and lot_type_cd = 'V' and yard_country_cd = 'USA'
and mbr_country = 'USA' AND mbr_mbrshp_type_cd IN ('BASIC', 'PREMIER')
  AND mbr_site_status_cd = 'A'
AND mbr_mbrshp_status_cd = 'A' AND mbr_lang_pref = 'en'
AND mbr_status = 'A'
AND yard_nbr in (180,43,97,186,10,7,59,367)
)
 select  f.buyer_nbr
 from filters f
"""

data8 = client.query(query8).to_dataframe()

In [25]:
data8['buyer_nbr'].nunique()

8179

In [26]:
data8.to_csv('data/raw/westlake_members.csv', index=False)

In [48]:
data8[data8['buyer_nbr']==718675]

,buyer_nbr
648167,718675
648407,718675
